# Linear Panel Estimations



In [53]:
import pandas as pd 
import numpy as np
import seaborn as sns
from numpy import linalg as la
from scipy.stats import chi2
from tabulate import tabulate
import LinearModelsProject1 as lm
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [ ]:

data = pd.read_csv("firms.csv")
N_list = data.firmid.unique()
T_list = data.year.unique()

N = data.firmid.unique().size
T = data.year.unique().size

y = data.ldsa.values.reshape((N*T,1))
l = data.lemp.values.reshape((N*T,1))
k = data.lcap.values.reshape((N*T,1))


constant = np.ones((y.shape[0], 1))
X = np.hstack([constant, l, k])



# Pooled Ordinary Least Squares (POLS)

We estimate the parameters in the following econometric model:

$\begin{equation} \log y_{it} = \beta_K\log k_{it} + \beta_L \log \ell_{it} + \log A_i + \log A_{it} \end{equation}$

Using the method of POLS.

In [55]:
## Assigning label names for the regressors and the dependent variable

label_x = ["c", "Log Employment", "Log Adjusted Capital"]
label_y = "Log Deflated Sales"

## Using the POLS method to estimate the parameters

ols_result = lm.estimate(y, X)

## Print the table containing estimation data

lm.print_table(
    (label_y, label_x), ols_result, title="Pooled OLS", floatfmt='.4f'
)


Pooled OLS
Dependent variable: Log Deflated Sales

                        Beta      Se    t-values
--------------------  ------  ------  ----------
c                     0.0000  0.0050      0.0000
Log Employment        0.6748  0.0102     66.4625
Log Adjusted Capital  0.3100  0.0091     33.9237
R² = 0.914
σ² = 0.131


# Fixed Effects Method (FE)

The fixed effects model is written as:

$$ \mathbf{\ddot{y}}_{it} = \mathbf{\ddot{X}}_{it}\beta + \mathbf{\ddot{u}}_{it} \tag{2}$$

Where the observations have been time-demeaned to get rid of the fixed effects. This is done by subtracting the time average of the observations, mathematically expressed as:

$$ \ddot{y}_{it} = y_{it} - \bar{y}_i\tag{3}$$

In [56]:
## Defining the demeaning matrix

def demeaning_matrix(T):
    Q_T = np.eye(T) - np.tile(1/T, (T, T))
    return Q_T

Q_T = demeaning_matrix(T)

## Apply Q_T on the observables

y_demean = lm.perm(Q_T, y)
x_demean = lm.perm(Q_T, X)

# Deleting the zero columns and updating the list of regressor labels

x_demean = x_demean[:, 1:]
label_x_fe = label_x[1:]

# Estimate using OLS for the FE method

fe_result = lm.estimate(
    y_demean, x_demean, transform='fe', T=T, robust_se=True
)

# Print the table containing estimation data

lm.print_table(
    (label_y,label_x_fe), 
    fe_result, title='FE regression', floatfmt='.4'
)




FE regression
Dependent variable: Log Deflated Sales

                        Beta       Se    t-values
--------------------  ------  -------  ----------
Log Employment        0.6942  0.04165      16.67
Log Adjusted Capital  0.1546  0.02995       5.163
R² = 0.477
σ² = 0.018


# First Differencing Method (FD)

We can write the FD-transformed regression model as:

$$ \Delta y_{it} = \Delta \mathbf{X}_{it}+\Delta u_{it} \tag{4}$$

Where the FD-transformation is denoted as:

$$ \Delta y_{it} = y_{it}-y_{it-1} \tag{5}$$

In [57]:
## Defining the FD matrix 

def fd_matrix(T):
    D_T = np.eye(T) - np.eye(T, k=-1)
    D_T = D_T[1:]
    return D_T

D_T = fd_matrix(T)

## Applying the FD matrix on the observables

y_diff = lm.perm(D_T, y)
x_diff = lm.perm(D_T, X[:,1:])

# Estimate using OLS for the first differencing method

fd_result = lm.estimate(y_diff, x_diff, transform='fd', T=T, robust_se=True)

# Print the table containing estimation data

lm.print_table(
    (label_y, label_x[1:]), 
    fd_result, title='FD regression', floatfmt='.4f'
)


FD regression
Dependent variable: Log Deflated Sales

                        Beta      Se    t-values
--------------------  ------  ------  ----------
Log Employment        0.5487  0.0284     19.3056
Log Adjusted Capital  0.0630  0.0229      2.7460
R² = 0.165
σ² = 0.014


# Random Effects Method (RE)

Random effects provides another approach of handling the unobserved heterogeneity. Instead of removing $c_i$ from the composite error term, we create a quasi-demeaned regression model denoted the following way:

$$ \check{y}_{it}=\mathbf{\check{X}}_{it}+\check{v}_{it}\tag{6}$$

Where the quasi-demeaned observables are obtained by:

$$ \check{y}_{it} = y_{it}-\hat{\lambda}\bar{y}_i \tag{7}$$

In [58]:
## Defining the matrix for between estimation

def mean_matrix(T):
    return np.tile(1/T, (1, T))
P_T = mean_matrix(T)

## Applying the mean matrix on the observables

y_mean = lm.perm(P_T, y)
x_mean = lm.perm(P_T, X)

## Estimating parameter values using the between estimation method

be_result = lm.estimate(
    y_mean, x_mean, transform='be', robust_se=True)

# Defining sigma squared values to obtain lambda

sigma_u = fe_result['sigma2']
sigma_c = be_result['sigma2'] - sigma_u/T
_lambda = 1 - np.sqrt(sigma_u/(sigma_u + T*sigma_c))

# Defining the quasi-demeaning matrix

C_t = np.eye(T) - _lambda*mean_matrix(T)

# Applying the matrix on the observables

x_re = lm.perm(C_t, X)
y_re = lm.perm(C_t, y)

# Estimating using OLS with the random effects method

re_result = lm.estimate(
    y_re, x_re, transform='re', T=T, robust_se=True
)

# Printing a table containing estimation data

lm.print_table(
    labels=(label_y, label_x), results=re_result, _lambda=_lambda,
    title='RE',
    floatfmt=['', '.4f', '.4f', '.2f']
)

RE
Dependent variable: Log Deflated Sales

                        Beta      Se    t-values
--------------------  ------  ------  ----------
c                     0.0000  0.0168        0.00
Log Employment        0.7197  0.0335       21.46
Log Adjusted Capital  0.1989  0.0261        7.62
R² = 0.642
σ² = 0.018
λ = 0.887


# SERIAL CORRELATION

We are interested in comparing the FE and FD methods. This can be done by a serial correlation test. The following algorithm is used to check if there are any observed serial correlation in the idiosyncratic error term:

In [59]:
## Updating the list of year to not only contain the unique values

T_list = data.year.values

## Filtering the list to exclude the even-year observations

T_list = np.delete(T_list, np.s_[::2], axis=0)

## Removing the first-year observations

reduced_T_list = T_list[T_list != 1969]

## Defining the function for serial correlation testing

def serial_corr(y, X, T, T_list):

    ## Defining estimated parameters and residuals

    b_hat = lm.est_ols(y, X)
    e = y - X@b_hat
    
    ## Defining the lag matrix

    L_T = np.eye(T, k=-1)
    L_T = L_T[1:]

    ## Applying the lag matrix
    
    e_l = lm.perm(L_T, e)
    
    ## Removing the first observation for every firm

    e = e[T_list != 1971]

    return lm.estimate(e, e_l)

In [60]:
## Generating serial correlation answers for FD and FE estimations

corr_result_fd = serial_corr(y_diff, x_diff, T-1, reduced_T_list)

corr_result_fe = serial_corr(y_demean, x_demean, T, T_list)

## Defining label and title names

label_ye = 'OLS residual, e\u1d62\u209c'
label_e = ['e\u1d62\u209c\u208B\u2081']
title = 'Serial Correlation'

## Printing the table for FD

lm.print_table(
    (label_ye, label_e), corr_result_fd, 
    title='Serial Correlation for First Differencing', floatfmt='.4f'
)

## Printing the table for FE

lm.print_table(
    (label_ye, label_e), corr_result_fe, 
    title='Serial Correlation for Fixed Effects', floatfmt='.4f'
)

IndexError: boolean index did not match indexed array along axis 0; size of axis is 4851 but size of corresponding boolean axis is 2205

# STRICT EXOGENEITY

We want to test if the assumption of strict exogeneity is satisfied in our model. If this is not the case, the consistency of FE, FD and RE will be flawed.

We can do this by adding a lead of our explanatory variables (lemp $\&$ lcap) to the model and use the corresponding t-values to determine significance.

In [ ]:
def exogeneity_test(X, y, T, T_list):

    ## Defining the lead matrix
    
    F_T = np.eye(T, k=1)
    F_T = F_T[:-1]

    ## Defining leads of the explanatory variables of interest

    capital_lead = lm.perm(F_T, X[:, 2].reshape(-1, 1))
    employment_lead = lm.perm(F_T, X[:, 1].reshape(-1, 1))

    ## Creating variable data for exogeneity test

    X_exo = X[T_list != 1979]
    X_exo = np.hstack((X_exo, capital_lead))
    X_exo = np.hstack((X_exo, employment_lead))
    y_exo = y[T_list != 1979]

    ## Defining the time-demeaning matrix

    Q_T = demeaning_matrix(T - 1)

    ## Applying the matrix

    yw_exo = lm.perm(Q_T, y_exo)
    xw_exo = lm.perm(Q_T, X_exo)
    xw_exo = xw_exo[:, 1:]

    ## Updating the list of x-labels

    label_exo = label_x_fe + ['Employment Lead'] + ['Capital Lead']

    # Estimate model using the fixed effects methods

    exo_test = lm.estimate(
        yw_exo, xw_exo, T=T - 1, transform='fe'
    )

    # Printing the table containing estimation data

    lm.print_table(
        (label_y, label_exo), 
        exo_test, title='Exogeneity test', floatfmt='.4f'
    )
    
exogeneity_test(X, y, T, T_list)

Exogeneity test
Dependent variable: Log Deflated Sales

                        Beta      Se    t-values
--------------------  ------  ------  ----------
Log Employment        0.6247  0.0290     21.5751
Log Adjusted Capital  0.0576  0.0253      2.2759
Employment Lead       0.1571  0.0296      5.3140
Capital Lead          0.0418  0.0279      1.5014
R² = 0.462
σ² = 0.016


# HAUSMAN TEST

As explained earlier in this file, the RE method takes has advantage over FE, as the unobserved heterogeneity isn't demeaned away. Since the RE also requires another assumption in $E[c_i\mathbf{x}_{it}] = 0$, we have to test if this assumption is met. This is what can be determined by a Hausman test.

Initially, we are checking for similarity in the estimates from the two different regression models to see if the coefficients are consistent. If not, we can assume that RE.1(b) ($E[c_i\mathbf{x}_{it}] = 0$) isn't satisfied.

In [ ]:
## Storing the beta estimates and the covariance matrix in variables

b_re = re_result['b_hat']
b_re = b_re[1:]

cov_re = re_result['cov']
cov_re = cov_re[1:,1:]

## Defining the differences between the coefficient estimates

hat_diff = fe_result['b_hat'] - b_re 

## Defining the differences between the covarinance matrices

cov_diff = fe_result['cov'] - cov_re

## Calculating the Hausman test value

H = hat_diff.T@la.inv(cov_diff)@hat_diff 

## Computing the p-value for the Hausman test

p_val = chi2.sf(H.item(), hat_diff.size)

In [ ]:
## Printing a table containing Hausman test results

def print_h_test(fe_result, re_result, hat_diff, p_val):
    table = []
    for i in range(len(hat_diff)):
        row = [
            fe_result['b_hat'][i], re_result['b_hat'][1:][i], hat_diff[i]
        ]
        table.append(row)

    print(tabulate(
        table, headers=['b_fe', 'b_re', 'b_diff'], floatfmt='.4f'
        ))
    print(f'The Hausman test statistic is: {H.item():.2f}, with p-value: {p_val:_.2f}.')
print_h_test(fe_result, re_result, hat_diff, p_val)

  b_fe    b_re    b_diff
------  ------  --------
0.7069  0.7331   -0.0262
0.1424  0.2148   -0.0724
The Hausman test statistic is: 15.86, with p-value: 0.00.


# WALD TEST

Our main interest was to analyze the dataset for constant returns to scale (CRS), which mathemathically can be shown to be the case where the exponents in the Cobb-Douglas function sums to 1.

Therefore, we've chosen to use a Wald test to investigate our hypothesis about CRS. We are using a Wald test for every regression model besides FD, since FE and FD should be consistent under the exact same econometric assumptions.

Our null- and alternative hypothesis is stated as:

$$ H_0 : \beta_K + \beta_L = 1 \tag{8}$$
$$ H_A : \beta_K + \beta_L \neq 1 \tag{9}$$

In [ ]:
## Defining variables for the covariance matrices and the coefficient estimates

cov_ols = ols_result['cov']
cov_fe = fe_result['cov']
cov_re = re_result['cov']

b_hat_ols = ols_result['b_hat']
b_hat_fe = fe_result['b_hat']
b_hat_re = re_result['b_hat']

## Hypothesis matrix R for testing beta_1 + beta_2 = 1

R = np.array([0, 1, 1])

## Reshaping R to fit the dimensions

R = np.reshape(R, (1,-1))

## Hypothesized value under H0

q0 = np.array([1])

## Computing the linear combination of estimates

q_hat_ols = np.dot(R, ols_result['b_hat'])
q_hat_fe = np.dot(R[:,1:], fe_result['b_hat'])
q_hat_re = np.dot(R, re_result['b_hat'])

## Reshaping the linear combinations

q_list = [q0, q_hat_ols, q_hat_fe, q_hat_re]

for q in q_list:
    q = np.reshape(q, (-1,1))

## Compute Wald statistic for the different methods
    
W = (q_hat_ols - q0).T @ la.inv(R @ cov_ols @ R.T) @ (q_hat_ols - q0)

W_fe = (q_hat_fe - q0).T @ la.inv(R[:,1:] @ cov_fe @ R[:,1:].T) @ (q_hat_fe - q0)

W_re = (q_hat_re - q0).T @ la.inv(R @ cov_re @ R.T) @ (q_hat_re - q0)

## Defining the degrees of freedoms for the different models

df_ols_re = 2
df_fe = 1

## Compute p-value for every model

p_value_ols = chi2.sf(W.item(), df_ols_re)
p_value_fe = chi2.sf(W_fe.item(), df_fe)
p_value_re = chi2.sf(W_re.item(), df_ols_re)

## Creating a table containing the Wald test results

test_results = [
    ["POLS", f"{W.item():.4f}", f"{p_value_ols:.6f}"],
    ["FE", f"{W_fe.item():.4f}", f"{p_value_fe:.6f}"],
    ["RE", f"{W_re.item():.4f}", f"{p_value_re:.6f}"]
]

headers = ["Method", "Wald Statistic", "P-value"]

print(tabulate(test_results, headers=headers, floatfmt=(".", ".4f", ".6f")))

Method      Wald Statistic    P-value
--------  ----------------  ---------
POLS                4.8971   0.086420
FE                 17.8704   0.000024
RE                 12.6369   0.001803


# Constant Returns to Scale (CRS)

At last, we have provided a simple table to show what the parameters from the different regression methods sums to.

In [ ]:
beta_sums = [
    ['POLS', ols_result['b_hat'][1]+ols_result['b_hat'][2]],
    [ 'RE' , re_result['b_hat'][1]+re_result['b_hat'][2]],
    [ 'FE' , fe_result['b_hat'][0]+fe_result['b_hat'][1]],
    [ 'FD' , fd_result['b_hat'][0]+fd_result['b_hat'][1]]
]

headers = ["Method", "Beta_L + Beta_K"]

print(tabulate(beta_sums, headers=headers, floatfmt=".4f"))



Method      Beta_L + Beta_K
--------  -----------------
POLS                 0.9867
RE                   0.9479
FE                   0.8493
FD                   0.7801
